In [14]:
import pandas as pd # used to create a database
import functools # used to maintain introspection on decorators
import requests
import urllib
import boto3 # used to access AWS resources
import time
import uuid # used to create a unique 'computer' id for each recipe
import json # used to store the scraped details
import os

from uuid import UUID # used to create a unique id for each recipe
from json import JSONEncoder # used to convert the UUID into a writable format
from urllib.request import Request, urlopen # used to download images
from sqlalchemy import create_engine # used to connect to the cloud

from sklearn.datasets import load_iris # used for learning and should be removed
#from decorators import noSuchElementException

from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC

#from selenium.webdriver.chrome.options import Options
#from webdriver_manager.chrome import ChromeDriverManager

DATABASE_TYPE = 'postgresql'
DBAPI = 'psycopg2'
ENDPOINT = 'datacollectionpipeline.c9nuz10uy6hn.us-east-1.rds.amazonaws.com' # Change it for your AWS endpoint
USER = 'postgres'
PASSWORD = 'Kilapila7'
PORT = 5432
DATABASE = 'postgres'

engine = create_engine(f"{DATABASE_TYPE}+{DBAPI}://{USER}:{PASSWORD}@{ENDPOINT}:{PORT}/{DATABASE}")
engine.connect()

data = load_iris()
iris = pd.DataFrame(data['data'], columns=data['feature_names'])
iris.head()

iris.to_sql('iris_dataset', engine, if_exists='replace')
df = pd.read_sql_table('iris_dataset', engine)
df.head()

s3_client = boto3.client('s3')
# (file_name, bucket, object_name)
response = s3_client.upload_file('raw_data/data.json', 'data-collection-pipeline-bucket', 'data.json') 
response = s3_client.upload_file('images/Strawberry Milk Bubble Tea/Strawberry Milk Bubble Tea 0.jpg.jpg', 'data-collection-pipeline-bucket', 'images')
#files = 'images/'
#for file in files:
    #imageJPG = os.path.join('images/', file)
    #s3_client.upload_file(imageJPG, 'data-collection-pipeline-bucket', 'images') #(directory, selected_text)

driver = webdriver.Chrome()

class data:
    '''
    This class contains all the attributes of the scraper

    Attributes:

    '''

    articles = [] # Used to make a list of recipes
    button = None # Used to interact with various button elements
    container = None # Used to store various container elements
    currentURL = "" # Used to store various urls 
    pages = [] # Used to append a list with pages links
    recipeLinks = [] # Used to store recipe links
    searchbar = None # Used to interact with search bar
    source = "" # Used to get page source code
    tag = None # Used to store various tag elements
    title = "" # Used to get the title
    totalPages = [] # Stores a list of pages

    # File Management
    count = 0 # Used in the creation of image filenames
    parentDirectory = "C:/Users/Millie/Documents/AiCore/AiCore/DataCollectionPipeline/" # Project directory
    dataDirectory = "" # Used to create folder
    imageDirectory = "" # Used to create folder 
    recipeDirectory = "" # Used to create modified folder names
    imageFileName = "" # Used to create image files

    # Scraped Information
    recipeDetails = {} # Used to store all the scraped recipe details
    imageScrapeLimiter = 0 # Used to limit the amount of times an image is scraped

    allergens = "" # Used to store scraped allergens
    alternatives = "" #Used to store scraped alternatives
    description = "" # Used to store the scraped description of the recipe
    freeFrom = "" # Used to store the scraped free from information
    imageLinks = [] # Used to scrape all of a recipes image links
    ingredients = "" # Used to store the scraped ingredients
    instructions = "" # Used to store scraped instructions
    mainPhoto = None # Used to store main photo link
    name = "" # Used to store scraped recipe name
    notes = "" # Used to store scraped recipe notes
    recipeTags = "" # Used to store scraped recipe tags
    storage = "" # Used to store scraped storage instructions
    timeCook = "" # Used to store scraped cook time
    timePrep = "" # Used to store scraped recipe  prep time 
    timeTotal = "" # Used to store scraped total time it takes to make the recipe

class scraper:
    '''
    This class is the main scraper. 

    This class will navigate through a webpage and scrape data into a json file and images into a folder separated into recipes.

    Attributes:
    '''

    def intitialize(self, url, searchTerm, delay):
        '''
        This function intitializes the scraper class.

        Arg:
            url (str): The target website
            searchTerm (str): The word we want to type into the search bar
            delay (int): The number of seconds used to delay a wait for elemnt and timeout exeptions

        Returns:

        '''
        global_ids = scraper.getUniqueID(scraper, 'https://www.pickuplimes.com/recipe/spicy-garlic-wok-noodles-213')
    
        self.getURL(self, url) # Have to start somewhere
        self.run(self)
        self.closeSession() # Have to end somewhere
    
    # DECORATORS

    def exceptionHandling(func):
        @functools.wraps(func) # maintains introspection
        def wrapper(*args, **kwargs):
            try:
                func(*args, **kwargs)
            except NoSuchElementException:
                print(f"{func.__name__} Exception: Element Not Found")
            except TimeoutException:
                print(f"{func.__name__} Exception: Timeout")
            return func(*args, **kwargs)
        return wrapper

    def functionTimer(func):
        """Prints the functions runtime"""
        @functools.wraps(func) # maintains introspection
        def wrapper(*args, **kwargs):
            start = time.perf_counter()
            func(*args, **kwargs)
            end = time.perf_counter()
            runtime = end - start
            print (f"Finished {func.__name__!r} in {runtime: .4f} secs")
            return func(*args, **kwargs)
        return wrapper

    # Best applied to small convenience functions that you don’t call directly yourself
    def debug(func):
        """Print the function signature and return value"""
        @functools.wraps(func) # maintains introspection
        def wrapper(*args, **kwargs):
            args_repr = [repr(a) for a in args]                     # Creates a list of positional arguments, repr() returns a string for each argument               
            kwargs_repr = [f"{k}={v!r}" for k, v in kwargs.items()] # Creates a list of keyword arguments, f-string formats each argument as key=value, !r specifier means that repr() is used to represent the value
            signature = ", ".join(args_repr + kwargs_repr)          # Join both lists together to make a signature
            print(f"Calling {func.__name__}({signature})")
            func(*args, **kwargs)
            print(f"{func.__name__!r} returned {func!r}")           
            return func
        return wrapper

    # Best applied to functions you only need to call less often
    def slowDown(func):
        '''Sleep before calling the function'''
        @functools.wraps(func) # maintains introspection
        def wrapper(*args, **kwargs):
            time.sleep(1)
            return func(*args, **kwargs)
        return wrapper

    def scrapeHandling(element):
        '''Wraps all scrape functions with exception handling'''
        def wrapperOuter(func):
            @functools.wraps(func) # maintains introspection
            def wrapperInner(*args, **kwargs):
                try:
                    func(*args, **kwargs)
                except NoSuchElementException:
                    #print(f"{func.__name__} Exception:", element, "Not Found")
                    element = "N/A"
                except TimeoutException:
                    #print(f"{func.__name__} Exception: Timeout")
                    element = "N/A"
                return func
            return wrapperInner
        return wrapperOuter

    def folderAlreadyExists(folderName) -> None:
        '''Handels the folder already exists exeption in the folder creation functions'''
        def wrapperOuter(func):
            @functools.wraps(func) # maintains introspection
            def wrapperInner(*args, **kwargs):
                try:
                    func(*args, **kwargs)
                except:
                    #print(f"{func.__name__} Error")
                    #print(folderName, "Folder Already Exists")
                    pass
                return func
            return wrapperInner
        return wrapperOuter
    
    def callCount(func) -> None:
        '''Counts the number of time the function has been called.'''
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            wrapper.calls += 1
            print(f"Call {wrapper.calls} of {func.__name__!r}")
            return func(*args, **kwargs)
        wrapper.calls = 0
        return wrapper

    def run(self) -> None:
        '''
        This function calls all the necissary functions in order.

        The order of function calls is for the class to navigate 
        through the website and scrape the text and images.

        Args:

        Returns:

        '''
        
        #self.acceptCookies()
        data.currentURL = self.findRecipeList(self)
        self.getAllRecipePages(self, data.currentURL)
        self.getRecipes(self, data.currentURL)
        self.cycleRecipeLinks(self)
        self.closeSession()   

    def cycleRecipeLinks(self) -> None:
        '''
        This function iterates though the list of recipe urls, passing each url to the makeImage() function.

        Args:

        Returns:

        '''

        for i in data.recipeLinks:
            data.currentURL = i
            self.makeImage(self, data.currentURL)

    #@callCount
    def getURL(self, url) -> None:
        '''
        Navigates to a website using a url passed as a perameter.
        
        Args:
            url (str): The target website
        
        Returns:

        '''

        driver.get(url) 

    def getTitle() -> None:
        '''
        Fetches the title.
        
        Args:
        
        Returns:
        
        '''

        data.title = driver.title

    def closeSession() -> None:
        '''
        Closes the driver
        
        Args:
        
        Returns:
        
        '''
        driver.quit()

    def getSourceCode() -> None:
        '''
        Fetches the current pages source code.
        
        Args:

        Returns:

        '''

        data.source = driver.page_source

    @exceptionHandling
    def search(self, searchTerm) -> None:
        '''
        Finds search bar and clicks it ready for input.
        
        Args:
            searchTerm (str): The word passed to the findSearchBar() fucntion that types into the search box
        
        Returns:

        '''

        # Find searchbar and click
        button = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, 'nav-searchbar-btn')))
        button.click()

        self.findSearchbar(self, searchTerm)

    def searchbarTextAndClick(searchTerm) -> None:
        '''
        This function types the searchTerm into the searchbar and presses enter which then navigates to the search results page.

        Args:
            searchTerm (str): The word to type into the search box

        Returns:
        
        '''

        try:
            data.searchbar.send_keys(searchTerm)
            data.searchbar.send_keys(Keys.RETURN) # Return = Enter
        except:
            print("Exception: No search term input")
    
    @exceptionHandling
    def findSearchbar(self, searchTerm) -> None:
        '''
        This function finds the searchbar and calls the searchbarTextAndClick() function with the searchTerm parameter.

        Args: 
            searchTerm (str): The word passed to the searchbarTextAndClick() fucntion that types into the search box

        Returns:
        
        '''

        data.searchbar = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.NAME, "sb")))
        self.searchbarTextAndClick(searchTerm)

    @exceptionHandling
    def home() -> None:
        '''
        Finds the title and clicks it.
        
        Args:
        
        Returns:
        
        '''

        title = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, 'nav-image')))
        title.click()

    @exceptionHandling
    def findRecipeList(self) -> None:
        '''
        Finds the recipe tab and clicks it.
        
        Args:
        
        Returns:
        
        '''

        data.button = WebDriverWait(driver,5).until(EC.presence_of_element_located((By.LINK_TEXT, 'Recipes')))
        data.button.click()

    @exceptionHandling
    def acceptCookies() -> None:
        '''
        Finds the accept cookies button and clicks it.
        
        Args:
        
        Returns:
        
        '''

        data.button = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '/html/body/div/div[2]/div[2]')))
        data.button.click()
    
    def getRecipes(self, url) -> None:
        '''
        Calls the functions to find the recipe container and puts all the recipes in a list.
        
        Args:
            url (str): The target website that contains all the recipe search results
        
        Returns:
        
        '''

        self.getRecipeContainer()
        self.makeRecipeList()

    @exceptionHandling
    def getRecipeContainer() -> None:
        '''
        This function finds the recipe container

        Args:

        Returns:

        '''

        data.container = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//*[@id='index-item-container']/div/div[2]/ul"))) 

    def makeRecipeList() -> None:
        '''
        This function finds the individual recipe page link and stores it in a list

        This function finds an individual recipes link by identifying the container 
        with all the recipes, then loops through that contain to find the individual 
        recipes and takes the url for that recipe and stores it in a list.
        
        Args:

        Returns:

        '''

        data.articles = data.container.find_elements(By.TAG_NAME, 'li')

        for i in data.articles:
            data.tag = i.find_element(By.TAG_NAME, 'a')
            data.recipeLinks.append(data.tag.get_attribute('href'))

    def getPageURL() -> None:
        '''
        Returns the current page url.
        
        Args:

        Returns:

        '''

        data.currentURL =  driver.current_url

    def getAllRecipePages(self, url) -> None:
        '''
        Calls the functions to navigate to each recipe page by modifying the current url and stores them in a list.
        
        Args:
            url (str): The target website that contains all the recipe search results
        
        Returns:
        
        '''

        self.getTotalPages()
        self.getSearchList()

    @exceptionHandling
    def getTotalPages() -> None:
        '''
        This function counts the total number of page results.

        Args:

        Returns:
        
        '''
        #totalPages = driver.find_element(By.CLASS_NAME, 'page-text') #actual
        data.totalPages = [1, 2, 3] #temp to shorten runtime

    def getSearchList() -> None:
        '''
        This function retrieves the url of each search result by modifying the url.

        Args:

        Returns:

        '''
        for i in data.totalPages:
            data.currentURL = driver.current_url
            url_change = "?page=" + str(i)
            next_page = data.currentURL + url_change
            data.pages.append(next_page)

    def getUniqueID(self, url) -> None:
        '''
        This function creates a uuid for each recipe by modifying  a url as a perameter. 

        Args: 
            url (str): The recipe page url

        Returns:

        '''
        
        page_ID = url
        just_ID = page_ID.replace(str("https://www.pickuplimes.com/recipe/"), "")

        ids = (just_ID, uuid.uuid4())

        return ids

    @scrapeHandling(data.name)
    def scrapeName() -> None:
        '''
        This function scrapes the name of the recipe. Exception handling is done with a decorator

        Args:

        Returns:

        '''

        data.name = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="header-info-col"]/div/header/h1'))).text

    @scrapeHandling(data.recipeTags)
    def scrapeTags() -> None:
        '''
        This function scrapes the tags of the recipe. Exception handling is done with a decorator

        Args:

        Returns:

        '''

        data.recipeTags = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="header-info-col"]/div/header/a[1]/div/p'))).text

    @scrapeHandling(data.description)
    def scrapeDescription() -> None:
        '''
        This function scrapes the description of the recipe. Exception handling is done with a decorator

        Args:

        Returns:

        '''

        data.description = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="header-info-col"]/div/header/span'))).text

    @scrapeHandling(data.timeTotal)
    def scrapeTotalTime() -> None:
        '''
        This function scrapes the total cook time of the recipe. Exception handling is done with a decorator

        Args:

        Returns:

        '''
        
        data.timeTotal = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="recipe-info-container"]/div[2]'))).text  

    @scrapeHandling(data.timePrep)
    def scrapePrepTime() -> None:
        '''
        This function scrapes the prep time of the recipe. Exception handling is done with a decorator

        Args:

        Returns:

        '''

        data.timePrep = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="recipe-info-container"]/div[3]'))).text

    @scrapeHandling(data.timeCook)
    def scrapeCookTime() -> None:
        '''
        This function scrapes the cook time of the recipe. Exception handling is done with a decorator

        Args:

        Returns:

        '''
        
        data.timeCook = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="recipe-info-container"]/div[4]'))).text

    @scrapeHandling(data.allergens)
    def scrapeAllergens() -> None:
        '''
        This function scrapes the allergens of the recipe. Exception handling is done with a decorator

        Args:

        Returns:

        '''

        data.allergens = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="allergen-info-container"]/div[1]/div'))).text 

    @scrapeHandling(data.alternatives)
    def scrapeAlternatives() -> None:
        '''
        This function scrapes the alternative ingrediants of the recipe. Exception handling is done with a decorator

        Args:

        Returns:

        '''
        
        data.alternatives = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="allergen-info-container"]/div[2]/div'))).text

    @scrapeHandling(data.freeFrom)
    def scrapeFreeFrom() -> None:
        '''
        This function scrapes what the recipe is free from. Exception handling is done with a decorator

        Args:

        Returns:

        '''
        
        data.freeFrom = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="allergen-info-container"]/div[3]/div'))).text 

    @scrapeHandling(data.ingredients)
    def scrapeIngredients() -> None:
        '''
        This function scrapes the recipe ingredients. Exception handling is done with a decorator

        Args:

        Returns:

        '''
        
        data.ingredients = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ingredient-direction-container"]/div/div[2]'))).text

    @scrapeHandling(data.instructions)
    def scrapeInstructions() -> None:
        '''
        This function scrapes the instructions for the recipe. Exception handling is done with a decorator

        Args:

        Returns:

        '''
        
        data.instructions = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ingredient-direction-container"]/div/div[4]/section/ol'))).text

    @scrapeHandling(data.notes)
    def scrapeNotes() -> None:
        '''
        This function scrapes the notes from the recipe. Exception handling is done with a decorator

        Args:

        Returns:

        '''
        
        data.notes = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ingredient-direction-container"]/div/div[4]/section/ul[1]/li'))).text

    @scrapeHandling(data.storage)
    def scrapeStorage() -> None:
        '''
        This function scrapes the storage instructions of the recipe. Exception handling is done with a decorator

        Args:

        Returns:

        '''
        
        data.storage = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ingredient-direction-container"]/div/div[4]/section/ul[2]/li'))).text

    @scrapeHandling(data.mainPhoto)
    def scrapeMainPhoto() -> None:
        '''
        This function scrapes the main image from the recipe. Exception handling is done with a decorator

        Args:

        Returns:

        '''
        
        data.mainPhoto = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="main-image-container"]/img')))


    @exceptionHandling
    def scrapeImages() -> None:
        '''
        This function scrapes the other images from the recipe. Exception handling is done with a decorator

        This function finds the image container then puts its children into a list. A limit is set based on the number of images found for later use. Finally the list is 
        iterated to get each images url link to create a list of image links

        Args:

        Returns:

        '''
        
        imageContainer = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="recipe-video"]/div[2]'))) # Find the container
        imageList = imageContainer.find_elements(By.XPATH, 'img') # Find the children
        data.imageScrapeLimiter = len(imageList)

        for i in imageList:
            link = i.get_attribute('src')
            data.imageLinks.append(link)

    @exceptionHandling
    def getRecipeDetails(self, url) -> None:
        '''
        This function calls the scrape functions

        This function navigates to a recipe page and calls all the scrape functions to collect 
        the data from the page. It the calls the function that stores all the data in a dictionary 
        and calls the function that writes that dictionary to a json file

        Args:
            url (str): The recipe page url
        
        Returns:

        '''
        self.getURL(self, url)

        self.scrapeName()  
        self.scrapeTags()
        self.scrapeDescription()
        self.scrapeTotalTime()
        self.scrapePrepTime()
        self.scrapeCookTime()
        self.scrapeAllergens()
        self.scrapeAlternatives()
        self.scrapeFreeFrom()
        self.scrapeIngredients()
        self.scrapeInstructions()
        self.scrapeNotes()
        self.scrapeStorage()
        self.scrapeMainPhoto()
        self.scrapeImages()

        self.storeDetails(self, url)
        self.jsonFile(self)

    def storeDetails(self, url) -> None:
        '''
        This function updates the data dictionarl with all the scraped information

        Args: 
            url (str): The recipe url to make a unique ID

        Returns:

        '''
        data.recipeDetails = {'ID': [], 'Name': [], 'Photo': [],'Tags': [], 'Description': [], 'Total Time': [], 'Prep Time': [], 'Cook Time': [], 'Allergens': [], 'Swaps': [], 'Free From': [], 'Ingredients': [], 'Directions': [], 'Notes': [], 'Storage': [], 'Images': []}
        data.recipeDetails['ID'].append(self.getUniqueID(self, url))
        data.recipeDetails['Name'].append(data.name)
        data.recipeDetails['Photo'].append(data.mainPhoto)
        data.recipeDetails['Tags'].append(data.recipeTags)
        data.recipeDetails['Description'].append(data.description)
        data.recipeDetails['Total Time'].append(data.timeTotal)
        data.recipeDetails['Prep Time'].append(data.timePrep)
        data.recipeDetails['Cook Time'].append(data.timeCook)
        data.recipeDetails['Allergens'].append(data.allergens)
        data.recipeDetails['Swaps'].append(data.alternatives)
        data.recipeDetails['Free From'].append(data.freeFrom)
        data.recipeDetails['Ingredients'].append(data.ingredients)
        data.recipeDetails['Directions'].append(data.instructions)
        data.recipeDetails['Notes'].append(data.notes)
        data.recipeDetails['Storage'].append(data.storage)
        data.recipeDetails['Images'].append(data.imageLinks)

    def jsonFile(self) -> None:
        '''This function creates a folder for the json file to be stored in
        
        This function creates a folder called 'raw_data' in the path for the 
        json file to be saved in. Uses a try except catch as it will throw an 
        error if the folder already exists.

        Args:

        Returns:
        
        '''

        self.makeRaw_DataFolder()

        # Deals with TypeError: Object of type UUID is not JSON serializable by encoding the UUID
        JSONEncoder_olddefault = JSONEncoder.default
        def JSONEncoder_newdefault(self, o):
            if isinstance(o, UUID): return str(o)
            return JSONEncoder_olddefault(self, o)
        JSONEncoder.default = JSONEncoder_newdefault

        self.jsonDump()

    @folderAlreadyExists("raw_data")
    def makeRaw_DataFolder():
        '''
        This function creates a folder.
        
        This function creates a folder for the json files to be stored in
        Throws an exception if the folder already exists which is handeled 
        with a decorator.

        Args:

        Returns:

        '''
        
        data.dataDirectory = "raw_data"
        parent_dir = "C:/Users/Millie/Documents/AiCore/AiCore/DataCollectionPipeline"
        path = os.path.join(parent_dir, data.dataDirectory)
        os.mkdir(path)
        print("Directory '% s' created" % data.dataDirectory)
    
    def jsonDump() -> None:
        '''This function writes the dictionary data to a json file
        
        This function stores data by writing the 'recipe_details' dictionary to a JSON file called 'data.json' in the folder just created
        The dicrionary is converted to a string using str() to deal with 'TypeError: Object of type WebElement is not JSON serializable
        
        Args:
        
        Returns:
        
        '''
        
        with open(os.path.join('raw_data', 'data.json'), 'w') as json_file:
            json.dump(str(data.recipeDetails), json_file)

    def downloadImage(self, url) -> None:
        '''
        This function creates a folder.
        
        This function calls the function that creates a folder called 'images' 
        Then calls the function that creates a folder named after the recipes name 
        Adds User-Agent Headers in a try/catch exeption handler to bypass 403 error
        Downloads the image into the folder of that recipe name
        
        Args:
        
        Returns:
        
        '''
        self.makeImagesFolder()
        self.makeRecipeFolder()

        try:
            # Adds headers to resolve 403 Fobidden Error
            opener=urllib.request.build_opener()
            opener.addheaders=[('User-Agent','Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1941.0 Safari/537.36')]
            urllib.request.install_opener(opener)
            path = os.path.join(data.recipeDirectory, data.imageFileName + '.jpg')
            urllib.request.urlretrieve(url, path)
        except:
            print("Error Downloading Images")         

    @folderAlreadyExists("Images")
    def makeImagesFolder() -> None:
        '''
        This function makes a folder.

        This function Makes a folder called images for the recipe images to be stored in.
        Uses a try except catch in a decorator as it will throw an error if the folders already exists.

        Args:

        Returns:

        '''
        
        data.imageDirectory = "images"
        path = os.path.join(data.parentDirectory, data.imageDirectory)
        os.mkdir(path)

    @folderAlreadyExists("Recipe")
    def makeRecipeFolder() -> None:
        '''
        This function makes a folder.

        This function Makes a folder named after the recipe for the images to be stored in.
        Uses a try except catch in a decorator as it will throw an error if the folders already exists.

        Args:

        Returns:

        '''
        
        data.recipeDirectory = "images/" + data.name
        path = os.path.join(data.parentDirectory, data.recipeDirectory)
        os.mkdir(path)
        print("Directory '% s' created" % data.recipeDirectory)

    def makeImage(self, url) -> None:
        '''
        This function makes a file name for the image and downloads it.
        
        This function retrieves the ID of each image using 'getRecipeDetails().
        Removes all unecissary elements from the ID string to create a file name.
        Pass the file name to 'downloadImages() to create a file.

        Args:
            url (str): The recipe url to scrape information from

        Returns:

        '''

        self.getRecipeDetails(self, url)

        for i in data.recipeDetails['Images']:

            for j in i:
                data.imageFileName = data.name + " " + str(data.count) + ".jpg"
                self.downloadImage(self, j)

                if data.count < data.imageScrapeLimiter:
                    data.count = data.count + 1
                else:
                    data.count = 0

scraper.intitialize(scraper, 'https://www.pickuplimes.com', 'lemons', 5)

# IDEALS ---------------------------------------------------------------------------------------------------------------
# link getTitle() and home()
# link getPageURL() and getUniqueID()
# get search results
# get search results on more than the first page
# make a separate main image
# counts how many recipes there are on the site
# a for loop that automaticaly changes the xpath to get all the images for the recipe
# replace all XPaths with written XPaths
# a method that makes a list containing all buttons and cycles through them looking for a link_text given as an argument
# seperate website specific methods from general functioning in a class
# make sure all code works with any website so is reusable and genralisable
# use the unique id (page url) to stop scraping recipes it has already scraped 
# have a wait for element function that takes a xpath and a perameter

KeyboardInterrupt: 

In [1]:
import os # for testing directories
import requests # for testing website response
import unittest
import uuid # for testing the recipe unique identifyer
from scraper import scraper

class scraperTestCase(unittest.TestCase):

    @classmethod
    def setUpClass(cls): # Runs at the begining of the file
        pass

    @classmethod
    def tearDownClass(cls): # Runs at the end of the file
        pass

    def setUp(self): # Runs before every test
        self.bot1 = scraper() 
        self.recipe = 'harissa-spiced-beans-898-0.jpg'
        self.directory = 'C:/Users/Millie/Documents/AiCore/AiCore/DataCollectionPipeline'
        self.friendlyID = 'harissa-spiced-beans-898'
        self.systemID = uuid.uuid4()
        self.dictionary = {"ID": []}
        self.number = str(3)
        self.recipeName = 'Harissa Spiced Beans'

        self.folderName1 = 'images'
        self.folderName2 = 'raw_data'
        self.imagePath = os.path.join(self.directory, self.folderName1)
        self.recipePath = os.path.join(self.directory, self.folderName1)
        self.dataPath = os.path.join(self.directory, self.folderName2)
        #self.handle = open("data.json", "r") # Dont forget to test the json file

    def tearDown(self): # Runs at the end of every test
        del self.bot1
    
    def test_dataClass(self):
        pass
    
    def test_decoratorClass(self):
        pass

    def test_exceptionHandling(self):
        pass

    def test_run(self):
        pass

    def test_cycleRecipeLinks(self):
        pass

    def test_getURL(self):
        print("test_getURL")
        response = requests.get('https://www.pickuplimes.com/')

        if response.ok:
            print("Website Found")
            return response.text
        else:
            print("!Website Not Found!")
            return 'Bad Response'

    def test_getTitle(self):
        pass

    def test_closeSession(self):
        pass

    def test_getSourceCode(self):
        pass

    def test_search(self):
        pass

    def test_searchbarTextAndClick(self):
        pass

    def test_findSearchbar(self):
        pass

    def test_home(self):
        pass

    def test_findRecipeList(self):
        pass

    def test_acceptCookies(self):
        pass

    def test_getRecipes(self):
        pass

    def test_getRecipeContainer(self):
        pass

    def test_makeRecipeList(self):
        pass

    def test_getPageURL(self):
        pass

    def test_getAllRecipePages(self):
        pass

    def test_getTotalPages(self):
        pass

    def test_getSearchList(self):
        pass

    def test_getUniqueID(self):
        pass

    def test_scrapeName(self):
        pass

    def test_scrapeTags(self):
        pass

    def test_scrapeDescription(self):
        pass

    def test_scrapeTotalTime(self):
        pass

    def test_scrapePrepTime(self):
        pass

    def test_scrapeCookTime(self):
        pass

    def test_scrapeAllergens(self):
        pass

    def test_scrapeAlternatives(self):
        pass

    def test_scrapeFreeFrom(self):
        pass

    def test_scrapeIngredients(self):
        pass

    def test_scrapeInstructions(self):
        pass

    def test_scrapeInstructions(self):
        pass

    def test_scrapeNotes(self):
        pass

    def test_scrapeStorage(self):
        pass

    def test_scrapeMainPhoto(self):
        pass

    def test_scrapeImages(self):
        pass

    def test_getRecipeDetails(self):
        pass

    def test_storeDetails(self):
        pass

    def test_jsonFile(self):
        pass

    def test_makeRaw_DataFolder(self):
        print('test_makeRaw_DataFolder')

        # Check directory path 
        self.assertEqual(self.dataPath, 'C:/Users/Millie/Documents/AiCore/AiCore/DataCollectionPipeline/raw_data')

        # Check file name
        self.assertEqual(os.path.join(self.recipePath, self.recipe), '')

        data.dataDirectory = "raw_data"
        
        '''
        path = os.path.join(parent_dir, data.dataDirectory)
        os.mkdir(path)
        print("Directory '% s' created" % data.dataDirectory)
        '''

        # UNFINISHED
    def jsonDump(self):
        print('jsonDump')

        # Check json file was created

        # Check dictionary was written in json
    
    # UNFINISHED
    def test_downloadImage(self):
        print('test_downloadImage')

        # Check directory path
        self.assertEqual(self.recipePath, 'C:/Users/Millie/Documents/AiCore/AiCore/DataCollectionPipeline\images')

        # Check image name
        self.assertEqual(self.recipeName, 'Harissa Spiced Beans') 

        # Check image was downloaded

    # UNFINISHED
    def test_makeImagesFolder(self):
        print('test_makeImagesFolder')       

        # Check directory is correct
        self.assertEqual(self.imagePath, 'C:/Users/Millie/Documents/AiCore/AiCore/DataCollectionPipeline\images') 

        # Check folder was created

    def test_makeRecipeFolder(self):
        print('test_makeRecipeFolder')

        # Check edited friendlyID
        self.assertEqual(self.recipe.replace(".jpg", "").replace("0", ""), 'harissa-spiced-beans-898-')

        # Check directory
        self.assertEqual(self.recipePath, 'C:/Users/Millie/Documents/AiCore/AiCore/DataCollectionPipeline\images')

    def test_makeImage(self):
        print("test_makeImage")

        self.dictionary['ID'].append(self.friendlyID)
        self.dictionary['ID'].append(self.systemID)

        # Check the dictionary contains the correct IDs
        self.assertEqual(self.dictionary['ID'][0], self.friendlyID)
        self.assertAlmostEqual(self.dictionary['ID'][1], self.systemID)

        # Check it removes all unnecessary bits from the string
        editedID = (self.dictionary['ID'][0])
        editedID = editedID.title()
        editedID = editedID.replace("-", " ")
        for i in editedID:
            if i.isdigit():
                editedID = editedID.replace(i , "")

        # Check the finished string is correct
        name = editedID + self.number + ".jpg"
        self.assertEqual(name, editedID + self.number + ".jpg")     

unittest.main(argv=['first-arg-is-ignored'], exit=False)

#if __name__ == "__main__":
#    test_scraper() 
#    print("Passed")

..................

test_downloadImage
test_getURL


......F.....................

Website Found
test_makeImage
test_makeImagesFolder
test_makeRaw_DataFolder
test_makeRecipeFolder



FAIL: test_makeRaw_DataFolder (__main__.scraperTestCase)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\Millie\AppData\Local\Temp\ipykernel_220\3239744656.py", line 175, in test_makeRaw_DataFolder
    self.assertEqual(self.dataPath, 'C:/Users/Millie/Documents/AiCore/AiCore/DataCollectionPipeline/raw_data')
AssertionError: 'C:/Users/Millie/Documents/AiCore/AiCore/DataCollectionPipeline\\raw_data' != 'C:/Users/Millie/Documents/AiCore/AiCore/DataCollectionPipeline/raw_data'
- C:/Users/Millie/Documents/AiCore/AiCore/DataCollectionPipeline\raw_data
?                                                               ^
+ C:/Users/Millie/Documents/AiCore/AiCore/DataCollectionPipeline/raw_data
?                                                               ^


----------------------------------------------------------------------
Ran 46 tests in 1.866s

FAILED (failures=1)
